# Writing Formatted Output

## Overview

### Questions

* How do I display status information during a simulation?
* How can I log user-defined quantities?
* How can I write formatted output to a text file?

### Objectives

* Demonstrate **user-defined log quantities**.
* Explain the use of **Table** to display status information during a simulation run.
* Show that **Table** can write to a file.

## Boilerplate code

In [1]:
import hoomd
import datetime

In [2]:
import os
fn = os.path.join(os.getcwd(), 'log.txt')
![ -e "$fn" ] && rm "$fn"

## Define the Simulation

This tutorial executes the Lennard-Jones particle simulation from a previous tutorial. 
See [*Introducing Molecular Dyamics*](../01-Introducing-Molecular-Dynamics/00-index.ipynb) for a complete description of this code.

In [3]:
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu)
sim.create_state_from_gsd(filename='../01-Introducing-Molecular-Dynamics/random.gsd')

integrator = hoomd.md.Integrator(dt=0.005)
cell = hoomd.md.nlist.Cell()
lj = hoomd.md.pair.LJ(nlist=cell)
lj.params[('A', 'A')] = dict(epsilon=1, sigma=1)
lj.r_cut[('A', 'A')] = 2.5
integrator.forces.append(lj)
nvt = hoomd.md.methods.NVT(kT=1.5, filter=hoomd.filter.All(), tau=1.0)
integrator.methods.append(nvt)
sim.operations.integrator = integrator

## Formatted output

The **Table** writer formats log quantities in human-readable text and writes them to `stdout` or a file.
**Table** only supports `scalar` and `string` quantities due to the limitations of this format.
This section shows you how to use **Table** to display status information during a simulation run.

## Add quantities to a Logger

The `categories` argument to **Logger** defines the **categories** that it will accept.

In [4]:
logger = hoomd.logging.Logger(categories=['scalar', 'string'])

Log the and simulation `timestep` and `tps` quantities:

In [5]:
logger.add(sim, quantities=['timestep', 'tps'])

You can also log user-defined quantities using functions, callable class instances, or class properties.
For example, this class computes the estimated time remaining:

In [6]:
class Status():
    def __init__(self, sim):
        self.sim = sim
    
    @property
    def seconds_remaining(self):
        try:
            return (self.sim.final_timestep - self.sim.timestep) / self.sim.tps
        except ZeroDivisionError:
            return 0        
    
    @property
    def etr(self):
        return str(datetime.timedelta(seconds=self.seconds_remaining))


Assign the loggable quantity using the tuple `(object, property_name, flag)`,  where `flag` is the string name of the flag for this quantity. (The tuple for callable objects would be `(callable, flag)`).

In [7]:
status = Status(sim)
logger[('Status', 'etr')] = (status, 'etr', 'string')

Represent the namespace of your user-defined quantity with a tuple of strings - `('Status', 'etr'`) above.
You can use any number of arbitrary strings in the tuple to name your quantity.

## Display quantities with Table

**Table** is a **Writer** that formats the quantities in a **Logger** into a human readable table.
Create one that triggers periodically:

In [8]:
table = hoomd.write.Table(trigger=hoomd.trigger.Periodic(period=5000),
                          logger=logger)

Add it to the simulation:

In [9]:
sim.operations.writers.append(table)

Run the simulation and see the output:

In [10]:
sim.run(100000)

Simulation.timestep  Simulation.tps     Status.etr   
       15000           5107.80649     0:00:18.598982 
       20000           5757.98828     0:00:15.630459 
       25000           6002.90400     0:00:14.159813 
       30000           6129.45818     0:00:13.051725 
       35000           6207.58806     0:00:12.081987 
       40000           6265.82958     0:00:11.171705 
       45000           6309.04420     0:00:10.302670 
       50000           6338.28260     0:00:09.466287 
       55000           6357.98123     0:00:08.650545 
       60000           6385.86845     0:00:07.829789 
       65000           6403.18069     0:00:07.027757 
       70000           6416.54075     0:00:06.233889 
       75000           6426.25893     0:00:05.446404 
       80000           6439.47447     0:00:04.658765 
       85000           6447.26919     0:00:03.877611 
       90000           6458.33434     0:00:03.096774 
       95000           6464.69062     0:00:02.320297 
      100000           6475.

Later in this notebook, you are going to create another **Table Writer**.
Remove `table` now to avoid confusing the two:

In [11]:
sim.operations.writers.remove(table)

## Save Table output to a file

**Table** writes to `stdout` by default.
It can write to a file (or any Python file-like object with `write` and `flush` methods) instead.

Open the file to write:

In [12]:
file = open('log.txt', 'x')

Create a **Table** **Writer** that outputs to this file and add it to the **Simulation**:

In [13]:
table_file = hoomd.write.Table(output=file,
                               trigger=hoomd.trigger.Periodic(period=5000),
                               logger=logger)
sim.operations.writers.append(table_file)

Run the simulation:

In [14]:
sim.run(100000)

You can read the file with standard tools:

In [15]:
!tail log.txt

      165000           6579.76985     0:00:06.839145 
      170000           6578.84926     0:00:06.080091 
      175000           6580.95965     0:00:05.318373 
      180000           6581.70999     0:00:04.558086 
      185000           6578.73154     0:00:03.800125 
      190000           6580.81394     0:00:03.039138 
      195000           6583.28617     0:00:02.278497 
      200000           6580.63825     0:00:01.519609 
      205000           6582.27364     0:00:00.759616 
      210000           6587.44309        0:00:00     


In this section, you have displayed **loggable quantities** during a simulation run and saved them to a text file.
This is the last section in the logging tutorial.

[Previous section](03-Storing-Particle-Shape.ipynb) / [Tutorial index](../README.md)